# 01 – Ingest EDGAR Emissions

This notebook wraps the reusable ingestion function in `src/pipeline/ingest_emissions.py` to tidy the EDGAR v8.0 NUTS2 workbook into a columnar dataset under `data/processed/`. Run the setup cell once to register the project path, then execute the ingestion cell to regenerate outputs when the source workbook changes.


In [1]:
from pathlib import Path
import sys

PROJECT_ROOT = Path.cwd().resolve()
if PROJECT_ROOT.name == "notebooks":
    PROJECT_ROOT = PROJECT_ROOT.parent
SRC_DIR = PROJECT_ROOT / "src"
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

PROJECT_ROOT


WindowsPath('C:/Users/narek.pirumyan/Desktop/IAE/2025/Big Data/Capstone Project/air-health-eu')

In [2]:
from pipeline.ingest_emissions import ingest_edgar_emissions

edgar_xlsx = PROJECT_ROOT / "DataSources" / "Emmissions" / "EDGARv8.0_GHG_by substance_GWP100_AR5_NUTS2_1990_2022.xlsx"
output_path = PROJECT_ROOT / "data" / "processed" / "emissions_nuts2.parquet"

output_path = ingest_edgar_emissions(edgar_xlsx, output_path)
output_path


WindowsPath('C:/Users/narek.pirumyan/Desktop/IAE/2025/Big Data/Capstone Project/air-health-eu/data/processed/emissions_nuts2.parquet')

In [3]:
import pandas as pd

preview = pd.read_parquet(output_path)
preview.head()


,nuts_id,nuts_label,country_iso,country_name,year,gas,sector,sector_group,emissions_kt_co2e
0,AT,None,AUT,Austria,1990,CO2,Dom_Avi,transport,64.581186
1,AT,None,AUT,Austria,1990,CO2,Dom_Ship,transport,28.087658
2,AT11,Burgenland,AUT,Austria,1990,CO2,Agriculture,agriculture,0.617001
3,AT11,Burgenland,AUT,Austria,1990,CO2,Buildings,buildings,443.808536
4,AT11,Burgenland,AUT,Austria,1990,CO2,Energy,energy,240.746162
